In [1]:
import tensorflow as tf
from tensorflow.python import autograph as ag

In [2]:
# Create a clean graph

mdl = tf.Graph()
with mdl.as_default():
    T = tf.placeholder(tf.float32, name = "T")
    Rval = tf.constant(286.0, name = "R_value") #for air
    R = tf.placeholder_with_default(Rval, Rval.shape, name = "R") #for air
    gammaval = tf.constant(1.4, name = "gamma_value") #for air
    gamma = tf.placeholder_with_default(gammaval, gammaval.shape, name = "gamma") #for air
    a = tf.sqrt(R*gamma*T)  

sess = tf.Session(graph = mdl)

# Initialize writer to allow visualization of model in TensorBoard
writer = tf.summary.FileWriter("log/example/a/", sess.graph)

# Close the writer
writer.close()

try:
    aval = sess.run(a, feed_dict={T : 300.0, gamma : 1.5})
    print(aval)
        
except tf.errors.InvalidArgumentError as e:
    print('Please provide value for : ' + e.node_def.name)
            
sess.close()

358.74783


In [3]:
inputVar=[
    {
      "name": "T",
      "type": "float"
    },
    {
      "name": "R",
      "type": "float",
      "value": 286.0
    },
    {
      "name": "gamma",
      "type": "float",
      "value": 1.4
    },
]

outputVar=[
    {
      "name": "speedOfSound",
      "type": "float"
    }
]
    

defaultValues = {}
for node in inputVar:
    if 'value' in node.keys():
        defaultValues[node['name']] = node['value']

print(defaultValues)

{'R': 286.0, 'gamma': 1.4}


In [4]:
def runSessionWithDefaults(sess, mdl, output, fd, defaultValues, defaultValuesUsed):
    try:
        aval = sess.run(output, feed_dict=fd)
    except tf.errors.InvalidArgumentError as e:
        #look for value available in default values
        if e.node_def.name in defaultValues.keys():
            tmp = mdl.get_tensor_by_name(e.node_def.name+':0')
            fd[tmp] = defaultValues[e.node_def.name]

            defaultValuesUsed[e.node_def.name] = defaultValues[e.node_def.name]

            aval, defaultValuesUsed = runSessionWithDefaults(sess, mdl, output, fd, defaultValues, defaultValuesUsed)
        else:
            print('Please provide value for: ' + e.node_def.name)
            aval = None
    return aval, defaultValuesUsed

In [5]:
def speedOfSound(invars):
    T = invars[0]
    R = invars[1]
    gamma = invars[2]
    a = tf.sqrt(R*gamma*T, name=outputVar[0]['name'])
    return a

# Create a clean graph
mdl = tf.Graph()
with mdl.as_default():
    invars = []
    for node in inputVar:
        tfType = tf.float32
        invars.append(tf.placeholder(tfType, name=node['name']))
    
    tfType = tf.float32
    tf_model = ag.to_graph(speedOfSound)
    output = tf_model(invars)
    tf.add_to_collection("output", output)

    for node in invars:
        tf.add_to_collection("input", node)  


evalInputVar=[
    {
        "name": "T",
        "type": "float",
        "value": 300.0
    },
    {
        "name": "R",
        "type": "float",
        "value": 300.0
    },
    {
        "name": "gamma",
        "type": "float",
        "value": 1.40
    }
]
sess = tf.Session(graph = mdl)

# Initialize writer to allow visualization of model in TensorBoard
writer = tf.summary.FileWriter("log/example/a/", sess.graph)

# Close the writer
writer.close()

fd = {}
for node in evalInputVar:
    fd[mdl.get_tensor_by_name(node['name']+':0')]=node['value']

defaultValuesUsed = {}
aval, defaultValuesUsed = runSessionWithDefaults(sess, mdl, output, fd, defaultValues, defaultValuesUsed)   

print('output: '+str(aval))
print('default values used: '+str(defaultValuesUsed))

sess.close()

output: 354.96478
default values used: {}


In [6]:
print(invars)
print(mdl.get_collection(name="input"))
print(mdl.get_collection(name="output"))

[<tf.Tensor 'T:0' shape=<unknown> dtype=float32>, <tf.Tensor 'R:0' shape=<unknown> dtype=float32>, <tf.Tensor 'gamma:0' shape=<unknown> dtype=float32>]
[<tf.Tensor 'T:0' shape=<unknown> dtype=float32>, <tf.Tensor 'R:0' shape=<unknown> dtype=float32>, <tf.Tensor 'gamma:0' shape=<unknown> dtype=float32>]
[<tf.Tensor 'speedOfSound/speedOfSound:0' shape=<unknown> dtype=float32>]


# Experiment with Node Dictionary

## 1. Existing model output is input of new model fragment

Initializing inputs, outputs, and methods for individual model fragments 

In [7]:
defaultValues = {}

inputVar1=[
    {
      "name": "T",
      "type": "float"
    },
    {
      "name": "R",
      "type": "float",
      "value": 286.0
    },
    {
      "name": "gamma",
      "type": "float",
      "value": 1.4
    },
]

outputVar1=[
    {
      "name": "speedOfSound",
      "type": "float"
    }
]

modelName1 = "speedOfSound"


for node in inputVar1:
    if 'value' in node.keys():
        defaultValues[node['name']] = node['value']
    
def speedOfSound(invars):
    T = invars[0]
    R = invars[1]
    gamma = invars[2]
    a = tf.sqrt(R*gamma*T, name=outputVar[0]['name'])
    return a

inputVar2=[
    {
        "name": "speedOfSound",
        "type": "float"
    },
    {
        "name": "objVelocity",
        "type": "float"
    }
]

outputVar2=[
    {
      "name": "machNumber",
      "type": "float"
    }
]

modelName2 = "machNumber"

def machNumber(invars):
    a = invars[0]
    v = invars[1]
    M = v/a
    return M

for node in inputVar2:
    if 'value' in node.keys():
        defaultValues[node['name']] = node['value']

In [8]:
# Create a clean graph
mdl = tf.Graph()
nodeDict = {}

with mdl.as_default():
    invars = []
    outvars = []
    
    for node in inputVar1:
        tfType = tf.float32
        tmp = tf.placeholder(tfType, name=node['name'])
        invars.append(tmp)
        node['graphRef'] = tmp
        nodeDict[node['name']] = node

    tf_model1 = ag.to_graph(speedOfSound)
    output = tf_model1(invars)
    
    for node in outputVar1:
        tfType = tf.float32
        tmp = output
        outvars.append(tmp)
        node['graphRef'] = tmp
        nodeDict[node['name']] = node
    
    for node in outvars:
        tf.add_to_collection("output", node)

    for node in invars:
        tf.add_to_collection("input", node)  
        
    invars = []
    outvars = []
    
    for node in inputVar2:
        if node['name'] in nodeDict.keys():
            tmp = nodeDict[node['name']]['graphRef']
            invars.append(tmp)
        else:
            tfType = tf.float32
            tmp = tf.placeholder(tfType, name=node['name'])
            invars.append(tmp)
            node['graphRef'] = tmp
            nodeDict[node['name']] = node

    tf_model2 = ag.to_graph(machNumber)
    output2 = tf_model2(invars)
    
    for node in outputVar2:
        tfType = tf.float32
        tmp = output2
        outvars.append(tmp)
        node['graphRef'] = tmp
        nodeDict[node['name']] = node
        
    for node in outvars:
        tf.add_to_collection("output", node)

    for node in invars:
        tf.add_to_collection("input", node)  


evalInputVar=[
    {
        "name": "T",
        "type": "float",
        "value": 300.0
    },
    {
        "name": "R",
        "type": "float",
        "value": 300.0
    },
    {
        "name": "gamma",
        "type": "float",
        "value": 1.40
    }
]
sess = tf.Session(graph = mdl)

# Initialize writer to allow visualization of model in TensorBoard
writer = tf.summary.FileWriter("log/example/a/", sess.graph)

# Close the writer
writer.close()

fd = {}
for node in evalInputVar:
    fd[mdl.get_tensor_by_name(node['name']+':0')]=node['value']

defaultValuesUsed = {}
aval, defaultValuesUsed = runSessionWithDefaults(sess, mdl, output, fd, defaultValues, defaultValuesUsed)   

print('output: '+str(aval))
print('default values used: '+str(defaultValuesUsed))

sess.close()

output: 354.96478
default values used: {}


In [9]:
nodeDict

{'T': {'name': 'T',
  'type': 'float',
  'graphRef': <tf.Tensor 'T:0' shape=<unknown> dtype=float32>},
 'R': {'name': 'R',
  'type': 'float',
  'value': 286.0,
  'graphRef': <tf.Tensor 'R:0' shape=<unknown> dtype=float32>},
 'gamma': {'name': 'gamma',
  'type': 'float',
  'value': 1.4,
  'graphRef': <tf.Tensor 'gamma:0' shape=<unknown> dtype=float32>},
 'speedOfSound': {'name': 'speedOfSound',
  'type': 'float',
  'graphRef': <tf.Tensor 'speedOfSound/speedOfSound:0' shape=<unknown> dtype=float32>},
 'objVelocity': {'name': 'objVelocity',
  'type': 'float',
  'graphRef': <tf.Tensor 'objVelocity:0' shape=<unknown> dtype=float32>},
 'machNumber': {'name': 'machNumber',
  'type': 'float',
  'graphRef': <tf.Tensor 'machNumber/truediv:0' shape=<unknown> dtype=float32>}}

Trying the naive approach for appending models

In [10]:
# Create a clean graph
mdl = tf.Graph()
nodeDict = {}

with mdl.as_default():
    invars = []
    outvars = []
    
    for node in inputVar1:
        tfType = tf.float32
        tmp = tf.placeholder(tfType, name=node['name'])
        invars.append(tmp)
        node['graphRef'] = tmp
        node['subModel'] = modelName1
        nodeDict[node['name']] = node

    tf_model1 = ag.to_graph(speedOfSound)
    output = tf_model1(invars)
    
    for node in outputVar1:
        tfType = tf.float32
        tmp = output
        outvars.append(tmp)
        node['graphRef'] = tmp
        node['subModel'] = modelName1
        nodeDict[node['name']] = node
    
    for node in outvars:
        tf.add_to_collection("output", node)

    for node in invars:
        tf.add_to_collection("input", node)  
        
    invars = []
    outvars = []
    
    for node in inputVar2:
        tfType = tf.float32
        tmp = tf.placeholder(tfType, name=node['name'])
        invars.append(tmp)
        node['graphRef'] = tmp
        node['subModel'] = modelName2
        nodeDict[node['name']] = node

    tf_model2 = ag.to_graph(machNumber)
    output2 = tf_model2(invars)
    
    for node in outputVar2:
        tfType = tf.float32
        tmp = output2
        outvars.append(tmp)
        node['graphRef'] = tmp
        node['subModel'] = modelName2
        nodeDict[node['name']] = node
        
    for node in outvars:
        tf.add_to_collection("output", node)

    for node in invars:
        tf.add_to_collection("input", node)  


sess = tf.Session(graph = mdl)

# Initialize writer to allow visualization of model in TensorBoard
writer = tf.summary.FileWriter("log/example/a/", sess.graph)

# Close the writer
writer.close()

#query 1
evalInputVar=[
    {
        "name": "objVelocity",
        "type": "float",
        "value": 1000.0
    },
    {
        "name": "T",
        "type": "float",
        "value": 300.0
    },
]

fd = {}
for node in evalInputVar:
    fd[nodeDict[node["name"]]["graphRef"]]=node['value']

defaultValuesUsed = {}
aval, defaultValuesUsed = runSessionWithDefaults(sess, mdl, output2, fd, defaultValues, defaultValuesUsed)   

print('output: '+str(aval))
print('default values used: '+str(defaultValuesUsed))

#query 2
evalInputVar=[
    {
        "name": "speedOfSound",
        "type": "float",
        "value": 350.0
    },
    {
        "name": "objVelocity",
        "type": "float",
        "value": 1000.0
    }
]

fd = {}
for node in evalInputVar:
    fd[nodeDict[node["name"]]["graphRef"]]=node['value']
    
defaultValuesUsed = {}
aval, defaultValuesUsed = runSessionWithDefaults(sess, mdl, output2, fd, defaultValues, defaultValuesUsed)   

print('output: '+str(aval))
print('default values used: '+str(defaultValuesUsed))

sess.close()

Please provide value for: speedOfSound_1
output: None
default values used: {}
output: 2.857143
default values used: {}


Append using Node Dictionary

In [11]:
# Create a clean graph
mdl = tf.Graph()
nodeDict = {}

with mdl.as_default():
    invars = []
    outvars = []
    
    for node in inputVar1:
        tfType = tf.float32
        tmp = tf.placeholder(tfType, name=node['name'])
        invars.append(tmp)
        node['graphRef'] = tmp
        node['subModel'] = modelName1
        nodeDict[node['name']] = node

    tf_model1 = ag.to_graph(speedOfSound)
    output = tf_model1(invars)
    
    for node in outputVar1:
        tfType = tf.float32
        tmp = output
        outvars.append(tmp)
        node['graphRef'] = tmp
        node['subModel'] = modelName1
        nodeDict[node['name']] = node
    
    for node in outvars:
        tf.add_to_collection("output", node)

    for node in invars:
        tf.add_to_collection("input", node)  
        
    invars = []
    outvars = []
    
    for node in inputVar2:
        if node['name'] in nodeDict.keys():
            tmp = nodeDict[node['name']]['graphRef']
            invars.append(tmp)
        else:
            tfType = tf.float32
            tmp = tf.placeholder(tfType, name=node['name'])
            invars.append(tmp)
            node['graphRef'] = tmp
            node['subModel'] = modelName2
            nodeDict[node['name']] = node

    tf_model2 = ag.to_graph(machNumber)
    output2 = tf_model2(invars)
    
    for node in outputVar2:
        tfType = tf.float32
        tmp = output2
        outvars.append(tmp)
        node['graphRef'] = tmp
        node['subModel'] = modelName2
        nodeDict[node['name']] = node
        
    for node in outvars:
        tf.add_to_collection("output", node)

    for node in invars:
        tf.add_to_collection("input", node)  


sess = tf.Session(graph = mdl)

# Initialize writer to allow visualization of model in TensorBoard
writer = tf.summary.FileWriter("log/example/a/", sess.graph)

# Close the writer
writer.close()

#query 1
evalInputVar=[
    {
        "name": "objVelocity",
        "type": "float",
        "value": 1000.0
    },
    {
        "name": "T",
        "type": "float",
        "value": 300.0
    },
]

fd = {}
for node in evalInputVar:
    fd[nodeDict[node["name"]]["graphRef"]]=node['value']

defaultValuesUsed = {}
aval, defaultValuesUsed = runSessionWithDefaults(sess, mdl, output2, fd, defaultValues, defaultValuesUsed)   

print('output: '+str(aval))
print('default values used: '+str(defaultValuesUsed))

#query 2
evalInputVar=[
    {
        "name": "objVelocity",
        "type": "float",
        "value": 1000.0
    }
]

fd = {}
for node in evalInputVar:
    fd[nodeDict[node["name"]]["graphRef"]]=node['value']
    
defaultValuesUsed = {}
aval, defaultValuesUsed = runSessionWithDefaults(sess, mdl, output2, fd, defaultValues, defaultValuesUsed)   

print('output: '+str(aval))
print('default values used: '+str(defaultValuesUsed))

sess.close()

output: 2.885309
default values used: {'R': 286.0, 'gamma': 1.4}
Please provide value for: T
output: None
default values used: {'R': 286.0, 'gamma': 1.4}


In [12]:
nodeDict

mdl.get_tensor_by_name("objVelocity:0")

<tf.Tensor 'objVelocity:0' shape=<unknown> dtype=float32>

## 2. Existing model input is output of new model fragment

In [13]:
defaultValues = {}

inputVar1=[
    {
      "name": "T",
      "type": "float"
    },
    {
      "name": "R",
      "type": "float",
      "value": 286.0
    },
    {
      "name": "gamma",
      "type": "float",
      "value": 1.4
    },
]

outputVar1=[
    {
      "name": "speedOfSound",
      "type": "float"
    }
]

modelName1 = "speedOfSound"


for node in inputVar1:
    if 'value' in node.keys():
        defaultValues[node['name']] = node['value']
    
def speedOfSound(invars):
    with tf.control_dependencies(invars):
        T = invars[0]
        R = invars[1]
        gamma = invars[2]
        a = tf.sqrt(R*gamma*T)
    return a

inputVar2=[
    {
        "name": "speedOfSound",
        "type": "float"
    },
    {
        "name": "objVelocity",
        "type": "float"
    }
]

outputVar2=[
    {
      "name": "machNumber",
      "type": "float"
    }
]

modelName2 = "machNumber"

def machNumber(invars):
    with tf.control_dependencies(invars):
        a = invars[0]
        v = invars[1]
        M = v/a
    return M

for node in inputVar2:
    if 'value' in node.keys():
        defaultValues[node['name']] = node['value']

In [14]:
# Create a clean graph
mdl = tf.Graph()
tf.reset_default_graph()
nodeDict = {}

with mdl.as_default():
    invars = []
    outvars = []
    
    for node in inputVar2:
        if node['name'] in nodeDict.keys():
            tmp = nodeDict[node['name']]['graphRef']
            invars.append(tmp)
        else:
            tfType = tf.float32
            tmp = tf.placeholder(tfType, name=node['name'])
            invars.append(tmp)
            node['graphRef'] = tmp
            node['subModel'] = modelName2
            nodeDict[node['name']] = node

    tf_model2 = ag.to_graph(machNumber)
    output2 = tf_model2(invars)
    
    for node in outputVar2:
        tfType = tf.float32
        tmp = output2
        outvars.append(tmp)
        node['graphRef'] = tmp
        nodeDict[node['name']] = node
    
    for node in outvars:
        tf.add_to_collection("output", node)

    for node in invars:
        tf.add_to_collection("input", node)  
        
    invars = []
    outvars = []
    
    for node in inputVar1:
        if node['name'] in nodeDict.keys():
            tmp = nodeDict[node['name']]['graphRef']
            invars.append(tmp)
        else:
            tfType = tf.float32
            tmp = tf.placeholder(tfType, name=node['name'])
            invars.append(tmp)
            node['graphRef'] = tmp
            node['subModel'] = modelName1
            nodeDict[node['name']] = node

    tf_model1 = ag.to_graph(speedOfSound)
    
    for node in outputVar1:
        if node['name'] in nodeDict.keys():
            print(node['name'])
            nodeDict[node['name']]['graphRef'] = tf_model1(invars)
            outvars.append(nodeDict[node['name']]['graphRef'])
            invars2 = []
            for node in inputVar2:
                if node['name'] in nodeDict.keys():
                    tmp = nodeDict[node['name']]['graphRef']
                    invars2.append(tmp)
                else:
                    tfType = tf.float32
                    tmp = tf.placeholder(tfType, name=node['name'])
                    invars2.append(tmp)
                    node['graphRef'] = tmp
                    node['subModel'] = modelName2
                    nodeDict[node['name']] = node
            output2 = tf_model2(invars2)
        else:
            tfType = tf.float32
            tmp = output1
            outvars.append(tmp)
            node['graphRef'] = tmp
            nodeDict[node['name']] = node
        
        
    for node in outvars:
        tf.add_to_collection("output", node)

    for node in invars:
        tf.add_to_collection("input", node)  


sess = tf.Session(graph = mdl)

# Initialize writer to allow visualization of model in TensorBoard
writer = tf.summary.FileWriter("log/example/a/", sess.graph)

# Close the writer
writer.close()

#query 1
evalInputVar=[
    {
        "name": "T",
        "type": "float",
        "value": 300.0
    },
    {
        "name": "objVelocity",
        "type": "float",
        "value": 1000.0
    }
]

fd = {}
for node in evalInputVar:
    fd[nodeDict[node["name"]]["graphRef"]]=node['value']

defaultValuesUsed = {}
aval, defaultValuesUsed = runSessionWithDefaults(sess, mdl, output2, fd, defaultValues, defaultValuesUsed)   

print('output: '+str(aval))
print('default values used: '+str(defaultValuesUsed))

#query 2
evalInputVar=[
    {
        "name": "objVelocity",
        "type": "float",
        "value": 900.0
    }
]

fd = {}
for node in evalInputVar:
    fd[nodeDict[node["name"]]["graphRef"]]=node['value']
    
defaultValuesUsed = {}
aval, defaultValuesUsed = runSessionWithDefaults(sess, mdl, output2, fd, defaultValues, defaultValuesUsed)   

print('output: '+str(aval))
print('default values used: '+str(defaultValuesUsed))

sess.close()

speedOfSound
output: 2.885309
default values used: {'R': 286.0, 'gamma': 1.4}
Please provide value for: T
output: None
default values used: {}


In [15]:
nodeDict

{'speedOfSound': {'name': 'speedOfSound',
  'type': 'float',
  'graphRef': <tf.Tensor 'speedOfSound_1/Sqrt:0' shape=<unknown> dtype=float32>,
  'subModel': 'machNumber'},
 'objVelocity': {'name': 'objVelocity',
  'type': 'float',
  'graphRef': <tf.Tensor 'objVelocity:0' shape=<unknown> dtype=float32>,
  'subModel': 'machNumber'},
 'machNumber': {'name': 'machNumber',
  'type': 'float',
  'graphRef': <tf.Tensor 'machNumber/truediv:0' shape=<unknown> dtype=float32>},
 'T': {'name': 'T',
  'type': 'float',
  'graphRef': <tf.Tensor 'T:0' shape=<unknown> dtype=float32>,
  'subModel': 'speedOfSound'},
 'R': {'name': 'R',
  'type': 'float',
  'value': 286.0,
  'graphRef': <tf.Tensor 'R:0' shape=<unknown> dtype=float32>,
  'subModel': 'speedOfSound'},
 'gamma': {'name': 'gamma',
  'type': 'float',
  'value': 1.4,
  'graphRef': <tf.Tensor 'gamma:0' shape=<unknown> dtype=float32>,
  'subModel': 'speedOfSound'}}

## Computational Graph Evolution

In [16]:
defaultValues = {}

inputVar_speedOfSound=[
    {
      "name": "T",
      "type": "float"
    },
    {
      "name": "R",
      "type": "float",
      "value": 286.0
    },
    {
      "name": "gamma",
      "type": "float",
      "value": 1.4
    },
]

outputVar_speedOfSound=[
    {
      "name": "speedOfSound",
      "type": "float"
    }
]

    
def speedOfSound(invars):
    with tf.control_dependencies(invars):
        T = invars[0]
        R = invars[1]
        gamma = invars[2]
        a = tf.sqrt(R*gamma*T)
    return a

inputVar_machNumber=[
    {
        "name": "speedOfSound",
        "type": "float"
    },
    {
        "name": "objVelocity",
        "type": "float"
    }
]

outputVar_machNumber=[
    {
      "name": "machNumber",
      "type": "float"
    }
]


def machNumber(invars):
    with tf.control_dependencies(invars):
        a = invars[0]
        v = invars[1]
        M = v/a
    return M

inputVar_gamma=[
    {
        "name": "Cp",
        "type": "float"
    },
    {
        "name": "Cv",
        "type": "float"
    }
]

outputVar_gamma=[
    {
      "name": "gamma",
      "type": "float"
    }
]


def gamma(invars):
    with tf.control_dependencies(invars):
        Cp = invars[0]
        Cv = invars[1]
        gamma = Cp/Cv
    return gamma

inputVar_pressure=[
    {
        "name": "rho",
        "type": "float"
    },
    {
        "name": "R",
        "type": "float",
        "value": 286.0
    },
    {
        "name": "T",
        "type": "float"
    }
]

outputVar_pressure=[
    {
      "name": "pressure",
      "type": "float"
    }
]


def pressure(invars):
    with tf.control_dependencies(invars):
        rho = invars[0]
        R = invars[1]
        T = invars[2]
        pressure = rho*R*T
    return pressure


In [17]:
def appendSubGraph(mdl, func, inputVars, outputVars, nodeDict, defaultValues):
    with mdl.as_default():
        invars = []
        outvars = []
        o = {}

        for node in inputVars:
            if node['name'] in nodeDict.keys():
                tmp = nodeDict[node['name']]['graphRef']
                invars.append(tmp)
            else:
                tfType = tf.float32
                tmp = tf.placeholder(tfType, name=node['name'])
                invars.append(tmp)
                node['graphRef'] = tmp
                node['subModel'] = func.__name__
                nodeDict[node['name']] = node

        tf_model = ag.to_graph(func)
        output = tf_model(invars)

        for node in outputVars:
            tfType = tf.float32
            tmp = output
            outvars.append(tmp)
            node['graphRef'] = tmp
            node['subModel'] = func.__name__
            nodeDict[node['name']] = node

        for node in outvars:
            tf.add_to_collection("output", node)

        for node in invars:
            tf.add_to_collection("input", node) 
        
        for node in inputVars:
            if 'value' in node.keys():
                defaultValues[node['name']] = node['value']
        
        o['mdl'] = mdl
        o['nodeDict'] = nodeDict
        o['defaultValues'] = defaultValues
        
    return o

In [18]:
# Create a clean graph
mdl = tf.Graph()
nodeDict = {}
defaultValues = {}

o = appendSubGraph(mdl, gamma, inputVar_gamma, outputVar_gamma, nodeDict, defaultValues)     
mdl = o['mdl']  
nodeDict = o['nodeDict']
defaultValues = o['defaultValues']

o = appendSubGraph(mdl, pressure, inputVar_pressure, outputVar_pressure, nodeDict, defaultValues)     
mdl = o['mdl']  
nodeDict = o['nodeDict']
defaultValues = o['defaultValues']

o = appendSubGraph(mdl, machNumber, inputVar_machNumber, outputVar_machNumber, nodeDict, defaultValues)     
mdl = o['mdl']  
nodeDict = o['nodeDict']
defaultValues = o['defaultValues']

o = appendSubGraph(mdl, speedOfSound, inputVar_speedOfSound, outputVar_speedOfSound, nodeDict, defaultValues)     
mdl = o['mdl']  
nodeDict = o['nodeDict']
defaultValues = o['defaultValues']

sess = tf.Session(graph = mdl)

# Initialize writer to allow visualization of model in TensorBoard
writer = tf.summary.FileWriter("log/example/a/", sess.graph)

# Close the writer
writer.close()

In [19]:
nodeDict['Cv'].keys()

dict_keys(['name', 'type', 'graphRef', 'subModel'])

In [20]:
defaultValues

{'R': 286.0, 'gamma': 1.4}

In [21]:
len(nodeDict)


10

## Append With Graph Rebuilding:

Case without rebuilding

In [22]:
def appendSubGraph(mdl, func, inputVars, outputVars, nodeDict, funcList, defaultValues):
    rebuildFlag = False
    with mdl.as_default():
        invars = []
        outvars = []
        o = {}

        for node in inputVars:
            if node['name'] in nodeDict.keys():
                tmp = nodeDict[node['name']]['graphRef']
                invars.append(tmp)
                node = nodeDict[node['name']]
                node['subModel'].append(func.__name__)
                nodeDict[node['name']] = node
            else:
                tfType = tf.float32
                tmp = tf.placeholder(tfType, name=node['name'])
                invars.append(tmp)
                node['graphRef'] = tmp
                node['subModel'] = list()
                node['subModel'].append(func.__name__)
                nodeDict[node['name']] = node

        tf_model = ag.to_graph(func)
        output = tf_model(invars)

        for node in outputVars:
            if node['name'] in nodeDict.keys():
                print('Option 2 encountered - rebuilding graph')
                rebuildFlag = True
                # Create a clean graph
                mdl = tf.Graph()
                # in funcList add entry of current function before any consumer function
                for index in range(len(funcList)):
                    if funcList[index]['name'] in node['subModel']:
                        break
                F = {}
                F['func'] = func
                F['name'] = func.__name__
                F['inputVar'] = inputVars
                F['outputVar'] = outputVars
                funcList.insert(index, F)
                mdl, nodeDict, funcList, defaultValues = appendSubGraphFromList(mdl, funcList, nodeDict = {}, funcList = [], defaultValues = {})
            else:
                tfType = tf.float32
                tmp = output
                outvars.append(tmp)
                node['graphRef'] = tmp
                node['subModel'] = list()
                node['subModel'].append(func.__name__)
                nodeDict[node['name']] = node

        for node in outvars:
            tf.add_to_collection("output", node)

        for node in invars:
            tf.add_to_collection("input", node) 

    if not rebuildFlag:
        F = {}
        F['func'] = func
        F['name'] = func.__name__
        F['inputVar'] = inputVars
        F['outputVar'] = outputVars
        funcList.append(F)
        for node in inputVars:
            if 'value' in node.keys():
                defaultValues[node['name']] = node['value']

    o['mdl'] = mdl
    o['nodeDict'] = nodeDict
    o['funcList'] = funcList
    o['defaultValues'] = defaultValues
        
    return o

In [23]:
def appendSubGraphFromList(mdl, funcSeqList, nodeDict, funcList, defaultValues):
    for f in funcSeqList:
        o = appendSubGraph(mdl, f['func'], f['inputVar'], f['outputVar'], nodeDict, funcList, defaultValues)     
        mdl = o['mdl']  
        nodeDict = o['nodeDict']
        funcList = o['funcList']
        defaultValues = o['defaultValues']
        #print(nodeDict)
    return mdl, nodeDict, funcList, defaultValues

In [24]:
# Create a clean graph
mdl = tf.Graph()
tf.reset_default_graph()
nodeDict = {}
defaultValues = {}
funcList = []

o = appendSubGraph(mdl, gamma, inputVar_gamma, outputVar_gamma, nodeDict, funcList, defaultValues)     
mdl = o['mdl']  
nodeDict = o['nodeDict']
defaultValues = o['defaultValues']
print(nodeDict)

o = appendSubGraph(mdl, speedOfSound, inputVar_speedOfSound, outputVar_speedOfSound, nodeDict, funcList, defaultValues)     
mdl = o['mdl']  
nodeDict = o['nodeDict']
defaultValues = o['defaultValues']
print(nodeDict) 

sess = tf.Session(graph = mdl)

# Initialize writer to allow visualization of model in TensorBoard
writer = tf.summary.FileWriter("log/example/a/", sess.graph)

# Close the writer
writer.close()

{'Cp': {'name': 'Cp', 'type': 'float', 'graphRef': <tf.Tensor 'Cp:0' shape=<unknown> dtype=float32>, 'subModel': ['gamma']}, 'Cv': {'name': 'Cv', 'type': 'float', 'graphRef': <tf.Tensor 'Cv:0' shape=<unknown> dtype=float32>, 'subModel': ['gamma']}, 'gamma': {'name': 'gamma', 'type': 'float', 'graphRef': <tf.Tensor 'gamma/truediv:0' shape=<unknown> dtype=float32>, 'subModel': ['gamma']}}
{'Cp': {'name': 'Cp', 'type': 'float', 'graphRef': <tf.Tensor 'Cp:0' shape=<unknown> dtype=float32>, 'subModel': ['gamma']}, 'Cv': {'name': 'Cv', 'type': 'float', 'graphRef': <tf.Tensor 'Cv:0' shape=<unknown> dtype=float32>, 'subModel': ['gamma']}, 'gamma': {'name': 'gamma', 'type': 'float', 'graphRef': <tf.Tensor 'gamma/truediv:0' shape=<unknown> dtype=float32>, 'subModel': ['gamma', 'speedOfSound']}, 'T': {'name': 'T', 'type': 'float', 'graphRef': <tf.Tensor 'T:0' shape=<unknown> dtype=float32>, 'subModel': ['speedOfSound']}, 'R': {'name': 'R', 'type': 'float', 'value': 286.0, 'graphRef': <tf.Tensor '

In [28]:
# Create a clean graph
mdl = tf.Graph()
tf.reset_default_graph()
nodeDict = {}
defaultValues = {}
funcList = []

funcSeqList = []

F = {}
F['func'] = gamma
F['inputVar'] = inputVar_gamma
F['outputVar'] = outputVar_gamma
funcSeqList.append(F)
F = {}
F['func'] = speedOfSound
F['inputVar'] = inputVar_speedOfSound
F['outputVar'] = outputVar_speedOfSound
funcSeqList.append(F)
F = {}
F['func'] = machNumber
F['inputVar'] = inputVar_machNumber
F['outputVar'] = outputVar_machNumber
funcSeqList.append(F)
F = {}
F['func'] = pressure
F['inputVar'] = inputVar_pressure
F['outputVar'] = outputVar_pressure
funcSeqList.append(F)


mdl, nodeDict, funcList, defaultValues = appendSubGraphFromList(mdl, funcSeqList, nodeDict, funcList, defaultValues)

sess = tf.Session(graph = mdl)

# Initialize writer to allow visualization of model in TensorBoard
writer = tf.summary.FileWriter("log/example/a/", sess.graph)

# Close the writer
writer.close()

In [26]:
# Create a clean graph
mdl = tf.Graph()
tf.reset_default_graph()
nodeDict = {}
defaultValues = {}
funcList = []

funcSeqList = []

F = {}
F['func'] = machNumber
F['inputVar'] = inputVar_machNumber
F['outputVar'] = outputVar_machNumber
funcSeqList.append(F)
F = {}
F['func'] = speedOfSound
F['inputVar'] = inputVar_speedOfSound
F['outputVar'] = outputVar_speedOfSound
funcSeqList.append(F)
F = {}
F['func'] = gamma
F['inputVar'] = inputVar_gamma
F['outputVar'] = outputVar_gamma
funcSeqList.append(F)

mdl, nodeDict, funcList, defaultValues = appendSubGraphFromList(mdl, funcSeqList, nodeDict, funcList, defaultValues)

sess = tf.Session(graph = mdl)

# Initialize writer to allow visualization of model in TensorBoard
writer = tf.summary.FileWriter("log/example/a/", sess.graph)

# Close the writer
writer.close()

Option 2 encountered - rebuilding graph
Option 2 encountered - rebuilding graph


In [27]:
funcList

[{'func': <function __main__.gamma(invars)>,
  'name': 'gamma',
  'inputVar': [{'name': 'Cp',
    'type': 'float',
    'graphRef': <tf.Tensor 'Cp:0' shape=<unknown> dtype=float32>,
    'subModel': ['gamma']},
   {'name': 'Cv',
    'type': 'float',
    'graphRef': <tf.Tensor 'Cv:0' shape=<unknown> dtype=float32>,
    'subModel': ['gamma']}],
  'outputVar': [{'name': 'gamma',
    'type': 'float',
    'graphRef': <tf.Tensor 'gamma/truediv:0' shape=<unknown> dtype=float32>,
    'subModel': ['gamma', 'speedOfSound']}]},
 {'func': <function __main__.speedOfSound(invars)>,
  'name': 'speedOfSound',
  'inputVar': [{'name': 'T',
    'type': 'float',
    'graphRef': <tf.Tensor 'T:0' shape=<unknown> dtype=float32>,
    'subModel': ['speedOfSound']},
   {'name': 'R',
    'type': 'float',
    'value': 286.0,
    'graphRef': <tf.Tensor 'R:0' shape=<unknown> dtype=float32>,
    'subModel': ['speedOfSound']},
   {'name': 'gamma',
    'type': 'float',
    'value': 1.4,
    'graphRef': <tf.Tensor 'gamma